In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

In [5]:
!tar xzvf /content/gdrive/MyDrive/darknet/cudnn/cudnn-10.1-linux-x64-v8.0.4.30.tgz -C /usr/local/

In [7]:
!cat /usr/local/cuda/include/cudnn_version.h | grep CUDNN_MAJOR -A 2

In [8]:
%cd /content/gdrive/MyDrive/darknet/bin/darknet

In [ ]:
!chmod +x ./darknet

In [9]:
%ls

In [ ]:
# Train
!LD_LIBRARY_PATH=/usr/local/cuda/lib64 ./darknet detector train train_2.1_600_frames_tuned_v4/r1mini.data train_2.1_600_frames_tuned_v4/yolov4.cfg -dont_show

In [ ]:
# mAP 확인
!LD_LIBRARY_PATH=/usr/local/cuda/lib64 ./darknet detector map train_2.1_600_frames_tuned_v4/r1mini.data train_2.1_600_frames_tuned_v4/yolov4.cfg train_2.1_600_frames_tuned_v4/backup/yolov4_6000.weights

In [ ]:
# video train 확인
!LD_LIBRARY_PATH=/usr/local/cuda/lib64 ./darknet detector demo train_2.1_600_frames_tuned_v4/r1mini.data train_2.1_600_frames_tuned_v4/yolov4.cfg train_2.1_600_frames_tuned_v4/backup/yolov4_4000.weights ./fastandfurious_short.mov -i 0 -out_filename 600fr_v4.mov -dont_show

In [ ]:
# ROI
!LD_LIBRARY_PATH=/usr/local/cuda/lib64 ./darknet detector test train_2/r1mini.data train_2/yolov3.cfg train_2/backup/yolov3_last.weights -ext_output train_2/r1mini/test.jpg

In [10]:
# 폴더에 있는 모든 img를 predict하기

# 변수 선언

#1. predict할 파일 들어있는 폴더 (txt파일에 입력될 path기 때문에 절대경로 추천, 마지막에 꼭 '/' 넣어줘야함.)
image_directory = '/content/gdrive/MyDrive/darknet/bin/darknet/train_2.1_600_frames_tuned_v4/final_test/'
#2. 확장자
extension = "*.jpg"
#3. txt파일 저장경로 및 파일 이름
save_at = './re.txt'
#4. predicted_pics 저장 경로+ 파일 이름 ex) predicted_img0.jpg,predicted_img1, ~ predicted_img10 로 저장할거면 아래처럼
predicted_pics_path ='/content/gdrive/MyDrive/darknet/bin/darknet/predicted_img/600_frames_v4_wt2000/'

# ***************************************************************************************************
# 아래는 걍 이것저것 편하게 넣어보려고 함수화한 것, 불필요하면 해당 자리에 그냥 사용하시는 커맨드 + $img_path 입력하면 됨 (.py로 선언 불가)
# ex) !./darknet detector test train/r1mini.data train/yolov3.cfg train/backup/yolov3_final.weights $img_path
#5. .data 파일 경로
data_path = '/content/gdrive/MyDrive/darknet/bin/darknet/train_2.1_600_frames_tuned_v4/r1mini.data'
#6. config 파일 경로
cfg_path = '/content/gdrive/MyDrive/darknet/bin/darknet/train_2.1_600_frames_tuned_v4/yolov4.cfg'
#7. weights 파일 경로
weights_path = '/content/gdrive/MyDrive/darknet/bin/darknet/train_2.1_600_frames_tuned_v4/backup/yolov4_2000.weights'

#command 함수
def darknet_predict_img(data_path,cfg_path,weights_path,img_path):
    !LD_LIBRARY_PATH=/usr/local/cuda/lib64 ./darknet detector test $data_path $cfg_path $weights_path $img_path


import glob
import PIL
import PIL.Image as Image
import make_path_txt
import os



d=0
with open(make_path_txt.make_data_path(image_directory,extension, save_at),'r') as fobj:
        image_List = [[num for num in line.split()] for line in fobj]
        for images in image_List:
            # print(images[0])
            img_path=images[0]
            darknet_predict_img(data_path,cfg_path,weights_path,img_path)
            #다크넷 경로 + prdictions.jpg
            predicted_image = Image.open(str(os.getcwd())+'/'+'predictions'+extension[1:])
            #predict 결과 저장할 경로+ 파일이름
            output = predicted_pics_path + '{}{}'.format(d,extension[1:])
            predicted_image.save(output) 
            d+=1

In [ ]:
# Train 중단뒤 다시 연결하기
!LD_LIBRARY_PATH=/usr/local/cuda/lib64 ./darknet detector train train_3/r1mini.data train_3/yolov3.cfg train_3/backup/yolov3_2000.weights -dont_show